# Example: Static inverse free-boundary equilibrium calculations (in ITER)

---

Here we will generate an equilibrium (find coil currents with the inverse solver) in an ITER-like tokamak. 

The machine description comes from files located [here](https://github.com/ProjectTorreyPines/FUSE.jl).

The equilbirium\profile parameters are **completely made up** - please experiment on your own and change them to more realistic values as you please!

### Import packages

In [1]:
import os
import matplotlib.pyplot as plt
import freegs4e
import numpy as np

### Create the machine object

In [2]:
# set paths
os.environ["ACTIVE_COILS_PATH"] = f"../machine_configs/ITER/ITER_active_coils.pickle"
os.environ["PASSIVE_COILS_PATH"] = f"../machine_configs/ITER/ITER_passive_coils.pickle"
os.environ["WALL_PATH"] = f"../machine_configs/ITER/ITER_wall.pickle"
os.environ["LIMITER_PATH"] = f"../machine_configs/ITER/ITER_limiter.pickle"

In [ ]:
# Now the machine can actually be built:
from freegsnke import build_machine
tokamak = build_machine.tokamak()

In [ ]:
# plot the machine
fig1, ax1 = plt.subplots(1, 1, figsize=(7, 15), dpi=120)
plt.tight_layout()

tokamak.plot(axis=ax1, show=False)
ax1.plot(tokamak.limiter.R, tokamak.limiter.Z, color='k', linewidth=1.2, linestyle="--")
# ax1.plot(tokamak.wall.R, tokamak.wall.Z, color='k', linewidth=1.2, linestyle="-")

ax1.grid(alpha=0.5)
ax1.set_aspect('equal')
ax1.set_xlim(1.1, 12.4)
ax1.set_ylim(-8.3, 8.3)
ax1.set_xlabel(r'Major radius, $R$ [m]')
ax1.set_ylabel(r'Height, $Z$ [m]')

### Instantiate an equilibrium

In [5]:
from freegsnke import equilibrium_update

eq = equilibrium_update.Equilibrium(
    tokamak=tokamak,       # provide tokamak object
    Rmin=1.1, Rmax=12.4,   # radial range
    Zmin=-8.3, Zmax=8.3,   # vertical range
    nx=129,                # number of grid points in the radial direction (needs to be of the form (2**n + 1) with n being an integer)
    ny=129,                # number of grid points in the vertical direction (needs to be of the form (2**n + 1) with n being an integer)
    # psi=plasma_psi
)

### Instantiate a profile object

In [6]:
# initialise the profiles
from freegsnke.jtor_update import ConstrainBetapIp
profiles = ConstrainBetapIp(
    eq=eq,        # equilibrium object
    betap=0.65,   # poloidal beta
    Ip=15e6,      # plasma current
    fvac=0.8,     # fvac = rB_{tor}
    alpha_m=2.0,  # profile function parameter
    alpha_n=1.0   # profile function parameter
)

### Load the static nonlinear solver

In [7]:
from freegsnke import GSstaticsolver
GSStaticSolver = GSstaticsolver.NKGSsolver(eq)    

### Constraints

In [8]:
import freegs4e

# set X-point locations
Rx = 4.93
Zx = -3.36
xpoints = [(Rx, Zx)]

# set any desired isoflux constraints with format (R1, Z1, R2, Z2), where (R1, Z1) and (R2, Z2) are 
# desired to be on the same flux contour.
isoflux = [
(Rx, Zx, 4.0228038405382+0.2, 0.5483452102114228),
# (Rx, Zx, 4.89677909,	-0.482538815),
(Rx, Zx, 4.254412439959347+0.1,	-1.448212877),
# (Rx, Zx, 4.557724156077329,	-2.43093497),
# (Rx, Zx, 6.225706428928894,	-2.841131482),
(Rx, Zx, 7.259463335861366,	-1.937088788),
(Rx, Zx, 7.911539671975528,	-0.767136564),
(Rx, Zx, 8.145511203289532,	 0.5532657790590028),
(Rx, Zx, 7.465729748693425,	 2.551811974626763),
(Rx, Zx, 6.052665521315404,	 3.743583690053864),
(Rx, Zx, 4.845816114370034,	3.779966078),
(Rx, Zx, 4.176404732054874+0.1,	 2.503053610591648),
]



           
# instantiate the constrain object
constrain = freegs4e.control.constrain(xpoints=xpoints,
                                         isoflux=isoflux,
                                       # psivals=psivals, # not used
                                         gamma=1e-12       # regularisation factor
                                         )

In [9]:
# import freegs4e

# # set X-point locations
# Rx = 5.15
# Zx = -3.3
# xpoints = [(Rx, Zx)]

# # set any desired isoflux constraints with format (R1, Z1, R2, Z2), where (R1, Z1) and (R2, Z2) are 
# # desired to be on the same flux contour.
# Rout = 8.1    # outboard midplane radius
# Rin = 4.25      # inboard midplane radius
# isoflux = [#(Rx,Zx, Rx,-Zx),     # link X-points
#            (Rout, 0, Rin, 0), # link inner and outer midplane points
#            (Rout, 0, Rx, Zx),   # link outer midplane point and X-point
#            (Rin, 0, Rx, Zx),   # link inner midplane point and X-point
#            (6.7, -2.5, Rx, Zx),   
#           #  (7.2, -1.25, Rx, Zx),   
#            (7.7, 2.5, Rx, Zx),   
#            (6.0, 3.75, Rx, Zx),   
#            (5.2, 4.0, Rx, Zx),   
#            (4.35, 2.5, Rx, Zx),
#           #  (4.3, -1.25, Rx, Zx),
#            (4.8, -2.5, Rx, Zx),   
#           #  (Rx, Zx, 4.2, -3.85),   
#           #  (Rx, Zx, 5.6, -4.7),   
# ]

           
# # instantiate the constrain object
# constrain = freegs4e.control.constrain(xpoints=xpoints,
#                                          isoflux=isoflux,
#                                        # psivals=psivals, # not used
#                                          gamma=1e-10       # regularisation factor
#                                          )

### The inverse solve

In [ ]:
GSStaticSolver.solve(eq=eq, 
                     profiles=profiles, 
                     constrain=constrain, 
                     target_relative_tolerance=1e-6,
                     verbose=True, # print output
                     picard=True, 
                     )

In [ ]:
fig1, ax1 = plt.subplots(1, 1, figsize=(7, 15), dpi=80)

ax1.grid(zorder=0, alpha=0.75)
ax1.set_aspect('equal')
eq.tokamak.plot(axis=ax1,show=False)                                                          # plots the active coils and passive structures
ax1.fill(tokamak.wall.R, tokamak.wall.Z, color='k', linewidth=1.2, facecolor='w', zorder=0)   # plots the limiter
eq.plot(axis=ax1,show=False)                                                                  # plots the equilibrium
constrain.plot(axis=ax1, show=False)                                                          # plots the contraints
ax1.set_xlim(1.1, 12.4)
ax1.set_ylim(-8.3, 8.3)


In [ ]:
eq.tokamak.getCurrents()

# # save coil currents to file
# import pickle
# with open('simple_diverted_currents_PaxisIp.pk', 'wb') as f:
#     pickle.dump(obj=inverse_current_values, file=f)